In [ ]:
! pip install transformers pip install git+https://github.com/huggingface/peft accelerate bitsandbytes

  Cloning https://github.com/huggingface/peft to /tmp/pip-req-build-5cszziss
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-5cszziss
  Resolved https://github.com/huggingface/peft to commit 096fe537370cf8a2cb55cc9bd05c7812ca919405
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.3 MB/s eta 0:00:00
  Created wheel for peft: filename=peft-0.8.2-py3-none-any.whl size=190936 sha256=f97822bd22cb655b14d7726c323fe5dd5d0dfabfd166e74d752e52b08e6828b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-ldbuzy23/wheels/4c/16/67/1002a2d4daa822eff130e6d85b90051b75d2ce0d26b9448e4a
Successfully built peft


In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
hinglish_peft_model_id = "smangrul/mistral_hinglish_instruct_poc"
mental_health_peft_model_id = "GRMenon/mental-health-mistral-7b-instructv0.2-finetuned-V2"

In [ ]:
config = PeftConfig.from_pretrained(hinglish_peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, load_in_4bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(hinglish_peft_model_id)

adapter_config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(32000, 4096)

In [ ]:
model = PeftModel.from_pretrained(model, hinglish_peft_model_id, adapter_name="hinglish")
_ = model.load_adapter(mental_health_peft_model_id, adapter_name="mental_health")

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

In [ ]:
list_of_adapters = ["hinglish", "mental_health"]
weights = [0.7, 0.3]
adapter_name = "mental_health_hinglish"
density = 0.7
combination_type="magnitude_prune"
model.add_weighted_adapter(list_of_adapters, weights, adapter_name, combination_type=combination_type, density=density)

In [ ]:
model.set_adapter("mental_health_hinglish")
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (hinglish): Dropout(p=0.1, inplace=False)
                  (mental_health): Dropout(p=0.05, inplace=False)
                  (mental_health_hinglish): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (hinglish): Linear(in_features=4096, out_features=16, bias=False)
                  (mental_health): Linear(in_features=4096, out_features=16, bias=False)
                  (mental_health_hinglish): Linear(in_features=4096, out_features=16, bias=False)
                )
     

In [ ]:
messages = [
    {"role": "user", "content": "Sad feelings ko kaise dur kare?"},
]
text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(text, return_tensors="pt")  # , add_special_tokens=False)
inputs = {k: v.to("cuda") for k, v in inputs.items()}
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    top_p=0.95,
    temperature=0.2,
    repetition_penalty=1.2,
    eos_token_id=tokenizer.eos_token_id,
)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


<s><s> [INST] WSad feelings ko kaise dur kare? [/INST]Sad feelings ko dur karne ke liye aapko apne dost aur family members se baat karen. Unhone aapko samaashta samajhane ka mauka dega. Aap bhi yoga aur meditation ki practice karte hain, jo aapki manav jankari aur samajhdar roop mein badhaye ga.</s>
